In [ ]:
from tams_pr2_guzheng.onset_to_path import OnsetToPath
from tams_pr2_guzheng.msg import RunEpisodeAction, RunEpisodeGoal, RunEpisodeResult, ExecutePathAction, ExecutePathGoal
from music_perception.msg import NoteOnset
from tams_pr2_guzheng.utils import row_from_result, stitch_paths, run_params

from tams_pr2_guzheng.paths import RuckigPath


In [2]:
import rospy
rospy.init_node('jupyter_playground')
import tf2_ros
import tf2_geometry_msgs
import actionlib

tf= tf2_ros.Buffer()
tf_listener= tf2_ros.TransformListener(tf)

In [ ]:
run_episode = actionlib.SimpleActionClient('run_episode', RunEpisodeAction)
run_episode.wait_for_server()

In [ ]:
execute_path = actionlib.SimpleActionClient('pluck/execute_path', ExecutePathAction)
execute_path.wait_for_server()

In [6]:
o2p= OnsetToPath()

In [ ]:
# run real episode
params = RuckigPath.random(note = 'd6', direction= 1.0, string_position= 0.05)
run_params(run_episode, params)
result = run_episode.get_result()
row = row_from_result(result)
o2p.add_sample(row)

In [7]:
# or add stub pluck
example_note = 'd6'
g = RunEpisodeResult()
g.parameters = RuckigPath.random(note= example_note, string_position= 0.1).action_parameters
o = NoteOnset()
o.confidence = 1.0
o.loudness = 6.0
o.note = example_note
g.onsets.append(o)
row = row_from_result(g)
o2p.add_sample(row)

In [8]:
p = o2p.get_path(note=example_note, loudness= 3.0)

In [ ]:
approach_path = copy.deepcopy(p)
approach_path.poses = approach_path.poses[0:1]
approach_pose = copy.deepcopy(approach_path.poses[0])
approach_pose.pose.position.z += 0.020
approach_path.poses.insert(0, approach_pose)

In [15]:
full_path = stitch_paths((p,p), tf)

In [ ]:
execute_path.send_goal(ExecutePathGoal(path= full_path, finger= 'ff'))
execute_path.wait_for_result()